In [12]:
import numpy as np
import pandas as pd
import tqdm

In [18]:
data_2018 = pd.DataFrame(pd.read_csv("data_2018.csv", encoding='euc-kr'))
data_2019 = pd.DataFrame(pd.read_csv("data_2019.csv", encoding='euc-kr'))
data_2020 = pd.DataFrame(pd.read_csv("data_2020.csv", encoding='euc-kr'))
data_2021 = pd.DataFrame(pd.read_csv("data_2021.csv", encoding='euc-kr'))
batter_data_2018 = pd.DataFrame(pd.read_csv("batter_data_2018.csv", encoding='euc-kr'))
batter_data_2019 = pd.DataFrame(pd.read_csv("batter_data_2019.csv", encoding='euc-kr'))
batter_data_2020 = pd.DataFrame(pd.read_csv("batter_data_2020.csv", encoding='euc-kr'))
batter_data_2021 = pd.DataFrame(pd.read_csv("batter_data_2021.csv", encoding='euc-kr'))

In [19]:
data = data_2018

In [29]:
data.rename(columns={'날짜': 'day'}, inplace=True)
data.head()

,name,day,상대,결과,타순,P,선발,타수,득점,안타,2타,3타,홈런,루타,타점,도루,도실,볼넷,사구,고4,삼진,병살,희타,희비,avg,출루,장타,OPS,투구,avLI,RE24,WPA,double
0,나성범,03-24,LG,W 4:2,3,RF,True,4,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0.000,0.000,0.000,0.000,12,1.49,-1.27,-0.100,0
1,나성범,03-25,LG,W 7:1,3,RF,True,3,1,1,0,0,0,1,0,1,0,0,2,0,0,0,0,0,0.143,0.333,0.143,0.476,15,0.90,1.42,0.113,0
2,나성범,03-27,한화,W 9:6,3,RF,True,4,1,2,0,0,1,5,2,0,0,0,0,0,1,0,0,0,0.273,0.385,0.545,0.930,15,0.93,4.22,0.329,0
3,나성범,03-28,한화,L 2:6,3,RF,True,4,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.200,0.294,0.400,0.694,9,0.87,-1.09,-0.087,0
4,나성범,03-29,한화,W 4:1,3,RF,True,4,0,2,0,0,0,2,0,0,0,0,0,0,2,0,0,0,0.263,0.333,0.421,0.754,15,1.60,-0.19,-0.076,0


In [26]:
# 더블헤더 처리
temp = np.zeros(len(data), dtype=np.int_)

# day, name이 같은 값이 있다는 것은 더블헤더 경기가 있다는것
# keep=False 옵션은 중복 값에 해당하는 튜플들을 모두 True로 리턴, 그 외 옵션 : 'first', 'last' 의 경우 둘 중 하나만 True
dup = data.loc[:,['day','name']].duplicated(keep=False) 

for i in range(len(data)):
  if dup[i]==False:
    temp[i]=0
  elif dup[i]==True:
    temp[i]=1

# 중복의 첫번째 행은 더블헤더 1차전
# 중복인 두번째 행이 더블헤더 2차전이다.
for i in range(1,len(temp)):
  if temp[i]==1 and temp[i-1]==1:
    temp[i]=2

data['double']=temp

In [28]:
data.head()

,name,day,상대,결과,타순,P,선발,타수,득점,안타,2타,3타,홈런,루타,타점,도루,도실,볼넷,사구,고4,삼진,병살,희타,희비,avg,출루,장타,OPS,투구,avLI,RE24,WPA,double
0,나성범,03-24,LG,W 4:2,3,RF,True,4,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0.000,0.000,0.000,0.000,12,1.49,-1.27,-0.100,0
1,나성범,03-25,LG,W 7:1,3,RF,True,3,1,1,0,0,0,1,0,1,0,0,2,0,0,0,0,0,0.143,0.333,0.143,0.476,15,0.90,1.42,0.113,0
2,나성범,03-27,한화,W 9:6,3,RF,True,4,1,2,0,0,1,5,2,0,0,0,0,0,1,0,0,0,0.273,0.385,0.545,0.930,15,0.93,4.22,0.329,0
3,나성범,03-28,한화,L 2:6,3,RF,True,4,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.200,0.294,0.400,0.694,9,0.87,-1.09,-0.087,0
4,나성범,03-29,한화,W 4:1,3,RF,True,4,0,2,0,0,0,2,0,0,0,0,0,0,2,0,0,0,0.263,0.333,0.421,0.754,15,1.60,-0.19,-0.076,0


In [36]:
#
# 선수마다 최근 5일의 타율을 구해야함. 
# 현재 day에서 -5부터 -1까지의 sum(hit)/sum(ab) 값을 구하면 된다

# hitter_data : 90명 타자 data
# record_data : 날짜별 타자 기록
# recent_day : 기본(최근 5일), 설정 가능
def make_recent_days(hitter_data, record_data, day=5):

  # 최근 5일(기본) 경기 타율 저장할 공간 생성
  recent_days_avg = np.zeros(len(record_data)) # 최근 5일(기본) 경기 타율
  recent_days_hitter = np.empty(len(record_data), dtype=np.chararray) # type : 문자열 저장을 위해 chararray
  recent_days_day = np.empty(len(record_data), dtype=object) # 기준이 되는 날짜
  recent_days_double = np.zeros(len(record_data), dtype=np.int8) # 더블헤더 분류 / 0 : 더블헤더 아님 / 1 : 더블헤더 1차전 / 2 : 더블헤더 2차전

  ps = 0
  for h in tqdm.tqdm(hitter_data['batter']): # 90명 타자 한명씩 접근
    hdata = record_data.loc[record_data['name']==h].reset_index() # 특정 타자 한명에 대해서만 데이터 추출
    for i in range(len(hdata['day'])): # 추출한 데이터에서 한 행씩 접근
      recent_days_hitter[ps] = h # 현재 타자
      r = hdata.loc[i,'day']  # 현재 기준 날짜
      recent_days_day[ps] = r
      d = hdata.loc[i,'double'] # 현재 더블헤더 분류
      recent_days_double[ps] = d      
      if d==2: # 더블헤더 2차전이라면
        hrdata = hdata.loc[hdata['day']>=r-day].loc[hdata['day']<=r].loc[hdata['double']!=2] # r 날짜까지 가져와서 2차전을 제외
      else: # 더블헤더가 아니고, 더블헤더 1차전이라면
        hrdata = hdata.loc[hdata['day']>=r-day].loc[hdata['day']<=r-1] # 접근한 행의 날짜를 기준으로 5일 경기
         
      if len(hrdata)==0: # 최근 5일 경기 없으면 넘어가고
        ps+=1
        continue
      else : # 있으면
        sum_ab = sum(hrdata['ab'])
        if sum_ab==0: # 경기 기록은 존재하지만 타수(ab)가 0
          ps+=1
          continue
        sum_hit = sum(hrdata['hit'])
        recent_days_avg[ps] = sum_hit/sum_ab
        ps+=1

  result_Dataframe = pd.DataFrame()
  result_Dataframe['day'] = recent_days_day
  result_Dataframe['name'] = recent_days_hitter
  result_Dataframe['recent_days_avg'] = recent_days_avg
  result_Dataframe['double'] = recent_days_double

  return result_Dataframe

In [31]:
#상대 타율 구하는 함수
#데이터 파일에서 vs 내림차순 후, 누적 타수 / 누적 안타 계산이 끝난 상태여야함
def make_vs_avg(hitter_data, record_data):
  vs_avg = np.zeros(len(record_data))
  name = np.empty(len(record_data), dtype=np.chararray) # type : 문자열 저장을 위해 chararray
  day = np.empty(len(record_data), dtype=np.int32) # 기준이 되는 날짜
  vs = np.empty(len(record_data), dtype=np.chararray)
  vs_ab = np.zeros(len(record_data))
  vs_hit = np.zeros(len(record_data))
  double_h = np.zeros(len(record_data))

  ps = 0
  for h in tqdm.tqdm(hitter_data['batter']): # 90명 타자 한명씩 접근
    hdata = record_data.loc[record_data['name']==h] # 특정 타자 한명에 대해서만 데이터 추출
    for r, d, v,ab, hit in zip(hdata['day'], hdata['double'], hdata['vs'], hdata['vs_cu_ab'], hdata['vs_cu_hit']): # 추출한 데이터에서 한 행씩 접근
      day[ps] = r  # 현재 기준 날짜
      name[ps] = h # 현재 타자
      vs[ps] = v
      vs_ab[ps] = ab
      vs_hit[ps] = hit
      double_h[ps] = d
      try:
        vs_avg[ps] = hit / ab
      except ZeroDivisionError:
        vs_avg[ps] = 0
      ps+=1
  
  result_Dataframe = pd.DataFrame()
  result_Dataframe['day'] = day
  result_Dataframe['double'] = double_h
  result_Dataframe['name'] = name
  result_Dataframe['vs_team'] = vs
  result_Dataframe['vs_ab'] = vs_ab
  result_Dataframe['vs_hit'] = vs_hit
  result_Dataframe['vs_avg'] = vs_avg
  result_Dataframe

  return result_Dataframe

In [34]:
batter_data_2018.rename(columns={'name': 'batter'}, inplace=True)
batter_data_2018

,Team,batter
0,NC,나성범
1,NC,스크럭스
2,NC,박민우
3,NC,노진혁
4,NC,권희동
...,...,...
85,한화,송광민
86,한화,정근우
87,한화,최재훈
88,한화,강경학


In [39]:
import datetime as dt

year=2018
dateString = str(year)+"-04-04"
dateFormatter = "%Y-%m-%d"
dt.datetime.strptime(dateString, dateFormatter)

for i in ragne(len(data)):
  
data['day']

0        03-24
1        03-25
2        03-27
3        03-28
4        03-29
         ...  
11311    10-06
11312    10-09
11313    10-19
11314    10-20
11315    10-23
Name: day, Length: 11316, dtype: object

In [37]:
recent_days = make_recent_days(batter_data_2018, data)

  0%|          | 0/90 [00:00<?, ?it/s]


TypeError: ignored

In [11]:
year=2018
import datetime as dt

dateString = str(year)+"-04-04"
dateFormatter = "%Y-%m-%d"
dt.datetime.strptime(dateString, dateFormatter)

datetime.datetime(2018, 4, 4, 0, 0)